# start point for MATSim work

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_05')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-03 11:16:55, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:16:55, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:16:55, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:16:55, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 06 Scenario

In [6]:
version_06_pickle_file_name = os.path.join(input_dir, 'working_scenario_06.pickle')
v_06_scenario = pickle.load(open(version_06_pickle_file_name, 'rb'))

In [7]:
print(v_06_scenario.road_net.links_df.shape)
print(v_06_scenario.road_net.nodes_df.shape)
print(v_06_scenario.road_net.shapes_df.shape)

(2190435, 44)
(688957, 12)
(1147816, 7)


## Remove centroid and centroid connectors from standard network
they were added to standard network via notebook 05 before applying external station project cards

In [8]:
v_06_scenario.road_net.links_df = v_06_scenario.road_net.links_df[
    ~(v_06_scenario.road_net.links_df.roadway.isin(["taz", "maz"]))
]

v_06_scenario.road_net.nodes_df = v_06_scenario.road_net.nodes_df[
    ~(v_06_scenario.road_net.nodes_df.model_node_id.isin(parameters.taz_N_list + parameters.maz_N_list))
]

v_06_scenario.road_net.shapes_df = v_06_scenario.road_net.shapes_df[
    (v_06_scenario.road_net.shapes_df.id.isin(v_06_scenario.road_net.links_df["id"]))
]

In [9]:
print(v_06_scenario.road_net.links_df.shape)
print(v_06_scenario.road_net.nodes_df.shape)
print(v_06_scenario.road_net.shapes_df.shape)

(1634769, 44)
(644475, 12)
(869970, 7)


## Make Travel Model Networks

In [15]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_06_scenario.road_net, 
    parameters = parameters)

2021-03-03 11:36:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:36:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:36:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 11:36:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [16]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(1634769, 44)
(644475, 12)
(869970, 7)


#probably not needed for MATSim
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

In [17]:
model_net.links_df.model_link_id.nunique()

1634769

In [18]:
model_net.nodes_df.model_node_id.nunique()

644475

In [19]:
model_net.shapes_df.id.nunique()

869970

In [23]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-03-03 12:13:35, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-03-03 12:13:35, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-03-03 12:13:35, INFO: Creating managed lane network.
2021-03-03 12:13:35, INFO: Creating managed lane network.
2021-03-03 12:13:35, INFO: Creating network with duplicated managed lanes
2021-03-03 12:13:35, INFO: Creating network with duplicated managed lanes
2021-03-03 12:13:35, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2021-03-03 12:13:35, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2021-03-03 12:13:36, INFO: Determining cntype
2021-03-03 12:13:36, INFO: Determining cntype
2021-03-03 12:13:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-03 12:13:36, INFO: 

In [24]:
model_net.links_mtc_df.shape

(1639168, 57)

In [25]:
model_net.links_mtc_df.model_link_id.nunique()

1639168

In [26]:
model_net.nodes_mtc_df.shape

(645980, 13)

In [27]:
model_net.nodes_df.model_node_id.nunique()

645980

## Write to Disk

## As shapefile

model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", 
                             "drive_access",
                            'lanes_EA', 'lanes_AM',
       'lanes_MD', 'lanes_PM', 'lanes_EV', "useclass_AM", "useclass_MD", 
                            "useclass_PM", "useclass_EV", "useclass_EA",
                            "cntype", "tollbooth", "tollseg"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

## As fixed width (for CUBE)

model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

%%capture
os.chdir(output_dir)
!make_cube.bat

%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s
!runtpp make_highway_ext_stn_connector_network.s

## As transit line file (for CUBE)

mtc.write_cube_fare_files(
    roadway_network=v_06_scenario.road_net,
    transit_network=v_06_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

standard_transit_net = StandardTransit.fromTransitNetwork(v_06_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

## As pickle

working_scenario_filename = os.path.join(output_dir, 'working_scenario_07.pickle')
pickle.dump(v_06_scenario, open(working_scenario_filename, 'wb'))

## As standard network

model_net.write(path = output_dir, filename = "v06")